In [3]:
from ast import literal_eval
import pandas as pd
import numpy as np
from sklearn.metrics import ndcg_score
from tqdm import tqdm
tqdm.pandas()

In [4]:
df_bert = pd.read_csv('data/id_bert_mmsr.tsv', sep='\t')
df_genres = pd.read_csv('data/id_genres_mmsr.tsv', sep='\t')
df_information = pd.read_csv('data/id_information_mmsr.tsv', sep='\t')
df_lyrics_ti = pd.read_csv('data/id_lyrics_tf-idf_mmsr.tsv', sep='\t')
df_lyrics_wv = pd.read_csv('data/id_lyrics_word2vec_mmsr.tsv', sep='\t')

In [5]:
df_bert.shape, df_genres.shape #, df_information.shape, df_lyrics_ti.shape, df_lyrics_wv.shape

((76115, 769), (76115, 2))

In [6]:
df_bert.head()

,id,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,9jbSytob9XRzwvB6,0.009225,0.041393,-0.003659,-0.030500,-0.006346,-0.019719,-0.075958,-0.003737,-0.000486,...,-0.037228,-0.027013,0.029167,0.009537,-0.010819,0.004054,-0.018671,0.012545,0.020696,-0.019794
1,Njp6JPM8vitbhVJU,0.000766,0.017016,0.016470,-0.033847,0.013241,0.012773,0.019069,-0.030150,0.002594,...,-0.018111,0.009309,0.069906,-0.042200,0.004261,-0.040005,-0.017133,0.005789,-0.041544,-0.010386
2,h48f46ZsT9h0Z5Dm,-0.015378,0.041258,-0.004096,-0.061025,-0.012853,0.028773,-0.045051,-0.016451,-0.007118,...,-0.042241,-0.014954,0.047022,-0.015661,-0.007859,0.016969,-0.021641,-0.003476,-0.012301,-0.022135
3,ZmXVK43zlqdeq6z8,0.030939,0.039541,0.006772,-0.024238,-0.014740,0.039660,-0.017606,-0.018583,-0.020654,...,-0.023624,0.012699,-0.018839,-0.018112,-0.010139,-0.042302,-0.043314,0.036310,-0.032837,-0.015104
4,PV5EXN6AIVBqvsLO,0.040497,0.061796,-0.017192,-0.012942,-0.040331,0.019082,0.001662,-0.036522,-0.022991,...,-0.025968,-0.023056,-0.019058,0.010935,-0.010183,-0.011044,-0.036080,-0.002666,-0.005304,-0.016923


In [7]:
df_genres.head()

,id,genre
0,0009fFIM1eYThaPg,['pop']
1,0010xmHR6UICBOYT,"['beats', 'underground hip hop', 'lo fi']"
2,002Jyd0vN4HyCpqL,"['hard rock', 'classic rock', 'rock', 'progres..."
3,006TYKNjNxWjfKjy,"['power metal', 'symphonic metal', 'symphonic ..."
4,007LIJOPQ4Sb98qV,"['post punk', 'new wave', 'dream pop', 'altern..."


In [8]:
#df_information.head()

In [9]:
#df_lyrics_ti.head()

In [10]:
#df_lyrics_wv.head()

In [11]:
if set(df_bert['id']) == set(df_genres['id']) == set(df_information['id']) == set(df_lyrics_ti['id']) == set(df_lyrics_wv['id']):
    print(True)

True


In [12]:
bert_embeddings = df_bert[df_bert.columns[1:]].values.tolist()
bert_embeddings = [str(i) for i in tqdm(bert_embeddings)]
new_bert_df = pd.DataFrame({'id': df_bert['id'],
                            'embeddings': bert_embeddings})

100%|██████████| 76115/76115 [00:29<00:00, 2609.34it/s]


In [13]:
new_bert_df['embeddings'] = new_bert_df['embeddings'].progress_apply(literal_eval)

100%|██████████| 76115/76115 [03:55<00:00, 323.54it/s]


In [14]:
#pickel# 

In [15]:
new_bert_df.head()

,id,embeddings
0,9jbSytob9XRzwvB6,"[0.0092247044667601, 0.041393231600523, -0.003..."
1,Njp6JPM8vitbhVJU,"[0.0007655373774468, 0.0170163195580244, 0.016..."
2,h48f46ZsT9h0Z5Dm,"[-0.0153776668012142, 0.0412583015859127, -0.0..."
3,ZmXVK43zlqdeq6z8,"[0.0309387072920799, 0.0395414233207702, 0.006..."
4,PV5EXN6AIVBqvsLO,"[0.0404967181384563, 0.0617959946393966, -0.01..."


In [20]:
def bert_similarity(v1, v2):
    return np.dot(v1, v2)

def get_most_similar_ids(id, search_df):
    emb = search_df[search_df['id'] == id]['embeddings']
    v1 = emb[emb.index[0]]
    search_df['similarities'] = search_df['embeddings'].apply(lambda v: bert_similarity(v, v1))
    search_df = search_df.sort_values(by='similarities', ascending=False)
    ids = search_df['id'].head(5).astype(str)
    ids = '||'.join(ids)
    # print(ids)
    return ids

In [23]:
df_bert_100 = df_bert.head(5)

In [24]:
# for id in df_bert['id']:
#     result = get_most_similar_ids(id, new_bert_df)
#     break

df_bert_100['similar_ids'] = df_bert_100['id'].progress_apply(lambda id: get_most_similar_ids(id, new_bert_df))

100%|██████████| 5/5 [00:40<00:00,  8.19s/it]
C:\Users\Johannes Schrott\AppData\Local\Temp\ipykernel_20920\4291705930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bert_100['similar_ids'] = df_bert_100['id'].progress_apply(lambda id: get_most_similar_ids(id, new_bert_df))


In [25]:
df_bert_100[['id', 'similar_ids']].shape#head()

(5, 2)

In [26]:
df_bert_100[['id', 'similar_ids']].head()

,id,similar_ids
0,9jbSytob9XRzwvB6,RsY9d2ha4UNV6ta6||Zo1FDQ8KBtxigWjN||yFJONyH8gT...
1,Njp6JPM8vitbhVJU,Njp6JPM8vitbhVJU||qv0QBR9djc9UhRaI||ofuF03vBR7...
2,h48f46ZsT9h0Z5Dm,LbMQ1RkMGYGc5Otf||W4E5681HWCRaJdAE||pFCYP9Ru3x...
3,ZmXVK43zlqdeq6z8,ZmXVK43zlqdeq6z8||PeANVvV2ktpUCjPf||AJzObB9wB9...
4,PV5EXN6AIVBqvsLO,PV5EXN6AIVBqvsLO||PCd7QZ7k44ODfPmU||31qALrrc1Y...


In [27]:
id2genre = dict(zip(df_genres['id'], df_genres['genre']))

In [28]:
df_genres.head()

,id,genre
0,0009fFIM1eYThaPg,['pop']
1,0010xmHR6UICBOYT,"['beats', 'underground hip hop', 'lo fi']"
2,002Jyd0vN4HyCpqL,"['hard rock', 'classic rock', 'rock', 'progres..."
3,006TYKNjNxWjfKjy,"['power metal', 'symphonic metal', 'symphonic ..."
4,007LIJOPQ4Sb98qV,"['post punk', 'new wave', 'dream pop', 'altern..."


In [29]:
df_bert_100_id_simid = df_bert_100[['id', 'similar_ids']]
id2simid = tuple(zip(df_bert_100_id_simid['id'], df_bert_100_id_simid['similar_ids']))

In [30]:
def get_binary_scores(id, similar_ids, id2genre):
    similar_ids_list = similar_ids.split('||')
    binary_result = []
    for similar_id in similar_ids_list:
        genres1 = literal_eval(id2genre[id])
        genres2 = literal_eval(id2genre[similar_id])
        intersect = [x for x in genres1 if x in genres2]
        if len(intersect) == 0:
            binary_result.append(0)
        else:
            binary_result.append(1)
    return binary_result

In [31]:
binary_similarity = []
for row in id2simid:
    # print(row)
    result = get_binary_scores(*row, id2genre)
    binary_similarity.append([row[0], row[1].split('||'), result])

In [33]:
n = 100
id_precision = []
for row in binary_similarity:
    values = row[2][:n]
    TP = sum(values)
    FP = len(values) - TP
    precision = TP / (TP + FP)
    id_precision.append([row[0], precision])

precision = np.mean([i[1] for i in id_precision])
print('Precision is:', precision)

first_occurences = []
for row in binary_similarity:
    try:
        first_occ = row[2][:n].index(1)+1
    except ValueError:
        continue
    else:
        first_occurences.append(first_occ)

MRR = sum([1/i for i in first_occurences]) / len(binary_similarity)
print('MRR is:', MRR)

y_true = []
y_pred = []
for row in binary_similarity:
    values = row[2][:n]
    y_pred.append(values)
    sorted_values = sorted(values)
    reversed_sorted_values = list(reversed(sorted_values))
    y_true.append(reversed_sorted_values)
y_true = np.array(y_true)
y_pred = np.array(y_pred)

print(y_true)
print(y_pred)

print('NDCG is:', ndcg_score(y_true, y_pred))

Precision is: 0.6
MRR is: 0.74
[[1 1 1 0 0]
 [1 1 1 1 1]
 [1 0 0 0 0]
 [1 1 1 1 0]
 [1 1 0 0 0]]
[[0 1 1 1 0]
 [1 1 1 1 1]
 [0 0 0 0 1]
 [1 1 1 0 1]
 [1 1 0 0 0]]
NDCG is: 0.8493251268266541
